In [1]:
import numpy as np

In [2]:
x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape

(1, 28, 28)

In [4]:
x[1].shape

(1, 28, 28)

In [5]:
x[0, 0]

array([[4.44520493e-01, 8.13515329e-01, 6.20054377e-01, 9.18629215e-01,
        2.75355539e-01, 1.14494911e-01, 8.45740180e-01, 3.08019249e-02,
        3.80022983e-01, 5.33451181e-01, 6.31186472e-01, 4.53361036e-01,
        4.96912391e-01, 7.03353176e-02, 2.24175743e-02, 3.49794445e-01,
        7.59428634e-01, 7.68351873e-01, 9.61228837e-01, 4.23678795e-01,
        8.75574759e-01, 7.22578343e-01, 9.49885582e-01, 2.81840475e-01,
        5.75938151e-01, 4.24761774e-01, 5.30141314e-01, 2.18215000e-01],
       [2.18964172e-01, 4.76710571e-01, 8.34697555e-01, 7.00216975e-01,
        1.14946635e-01, 1.84989591e-01, 7.39482760e-01, 6.74989534e-01,
        4.46461270e-02, 6.21389763e-01, 1.97659786e-01, 2.48111139e-01,
        9.81701645e-01, 3.09441783e-01, 9.16921030e-01, 2.22793193e-01,
        2.42825065e-01, 3.89313816e-01, 1.02824351e-01, 7.99649238e-01,
        2.46328756e-01, 6.92935174e-01, 6.76320692e-01, 6.76236083e-01,
        4.45990466e-01, 4.70439200e-01, 7.51651003e-01, 9.10329

이게 뭐지

7.4.3 합성곱 계층 구현하기

In [7]:
im2col(input_data, filter_h, filter_w, stride=1, pad=0)

NameError: name 'im2col' is not defined

In [8]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

ModuleNotFoundError: No module named 'common'

In [9]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w -1).transpose(0, 3, 1, 2)
        
        return out

In [10]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        out = np.max(col, axis=1)
        
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

7.5 CNN 구현하기

In [11]:
class SimpleConvNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param={'filter_num':30, 'filter_size':5,
                           'pad':0, 'stride':1},
                hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
                            filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) *
                              (conv_output_size/2))

In [12]:
self.params = {}
self.params['W1'] = weight_init_std * \
                    np.random.randn(filter_num, input_dim[0],
                                   filter_size, filter_size)
self.params['b1'] = np.zeros(filter_num)
self.params['W2'] = weight_init_std * \
                    np.random.randn(pool_output_size,
                                   hidden_size)
self.params['b2'] = np.zeros(hidden_size)
self.params['W3'] = weight_init_std * \
                    np.random.randn(hidden_size, output_size)
self.params['b3'] = np.zeros(output_size)

NameError: name 'self' is not defined

In [13]:
self.layers = OrderedDict()
self.layers['Conv1'] = Convolution(self.params['W1'],
                                  self.params['b1'],
                                  conv_param['stride'],
                                  conv_param['pad'])
self.layers['Relu1'] = Relu()
self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
self.layers['Affline1'] = Affline(self.params['W2'],
                                 self.params['b2'])
self.layers['Relu2'] = Relu()
self.layers['Affline2'] = Affline(self.params['W3'],
                                 self.params['b3'])

self.last_layer = softmaxWithLoss()

NameError: name 'OrderedDict' is not defined

In [14]:
def predict(self, x):
    for layer in self.layers.values():
        x = layer.forward(x)
    return x

def loss(self, x, t):
    y = self.predict(x)
    return self.last_layer.forward(y, t)

In [15]:
def gradient(self, x, t):
    self.loss(x, t)
    
    dout = 1
    dout = self.last_layer.backward(dout)
    
    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
        dout = layer.backward(dout)
        
    grads = {}
    grads['W1'] = self.layers['Conv1'].dW
    grads['b1'] = self.layers['Conv1'].db
    grads['W2'] = self.layers['Affline1'].dW
    grads['b2'] = self.layers['Affline1'].db
    grads['W3'] = self.layers['Affline2'].dW
    grads['b3'] = self.layers['Affline2'].db
    
    return grads